# You have this week to finish this coding competition. Result will be announced on Monday.

***

## Tasks 
    [Note to self : Complete the tasks and git commit :P]

   [x] Make a .csv file based on Hypopnea from data/apnea/data.xml (previously mesa-sleep-0001-profusion.xml)

   [x] Convert the given mesa-sleep-0001.edf file into a csv file

   [x] Make an analogy from .edf file to understand which column is flow(Flow data is sampled at 32 samples per second)

   [x] Add the first column as time or Epoch, ime should start from 0 with consecutive steps of 1/32 second (see the output file requirement) 

   [x] Export label/EventType Stages|Stages which has a stage, startTime, and duration of a stage as csv (see the output file format.)
   
   [ ] Combine all these three files together in one file (like output/combined/mesa-sleep-0001.csv)
   
   
   
### lots of work
   
![alt text](https://media.giphy.com/media/5dlWYlh44HeQ8/giphy.gif "AAAAAAAAAAAA")

Let's get down to work..

## Task 1 @ Data processing data/apnea/data.xml

In [1]:
import xml.etree.cElementTree as et
import pandas as pd

/home/courage/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/courage/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
def getvalueofnode(node):
    # return node text or None
    return node.text if node is not None else None

In [3]:
parsed_xml = et.parse("data/apnea/data.xml")
parsed_xml

In [4]:
dfcols = ['Name', 'Start', 'Duration', 'Input']
data = pd.DataFrame(columns=dfcols)
root = parsed_xml.getroot()
root.tag

'CMPStudyConfig'

In [5]:
for node in root.iter("ScoredEvent"):
    name = node.find('Name')
    start = node.find('Start')
    duration = node.find('Duration')
    iput = node.find('Input')

    if(getvalueofnode(name) == "Hypopnea"):
        data = data.append(
        pd.Series([getvalueofnode(name), getvalueofnode(start), getvalueofnode(duration),
                   getvalueofnode(iput)], index=dfcols),ignore_index=True)

data.to_csv('out.csv', index=False)

## It works!!!!

![alt text](https://media.giphy.com/media/QFz0Acu7er6hy/giphy.gif "Yes Did it")

### Attributions 
 The Python Documentation on [The ElementTree XML API](https://docs.python.org/2/library/xml.etree.elementtree.html "Why Just Hover?? The Documentation rocks")

## Task 2 @ Processing signals from data/flow/mesa-sleep-0001.edf 

In [6]:
import pyedflib
import numpy as np

f = pyedflib.EdfReader("data/flow/mesa-sleep-0001.edf")
n = f.signals_in_file
signal_labels = f.getSignalLabels()
print(signal_labels)

['EKG', 'EOG-L', 'EOG-R', 'EMG', 'EEG1', 'EEG2', 'EEG3', 'Pres', 'Flow', 'Snore', 'Thor', 'Abdo', 'Leg', 'Therm', 'Pos', 'EKG_Off', 'EOG-L_Off', 'EOG-R_Off', 'EMG_Off', 'EEG1_Off', 'EEG2_Off', 'EEG3_Off', 'Pleth', 'OxStatus', 'SpO2', 'HR', 'DHR']


In [7]:
print("edfsignals: %i" % f.signals_in_file)
print("file duration: %i seconds" % f.file_duration)
print("startdate: %i-%i-%i" % (f.getStartdatetime().day,f.getStartdatetime().month,f.getStartdatetime().year))
print("starttime: %i:%02i:%02i" % (f.getStartdatetime().hour,f.getStartdatetime().minute,f.getStartdatetime().second))

edfsignals: 27
file duration: 43199 seconds
startdate: 1-1-1985
starttime: 20:29:59


In [8]:
print("datarecord duration: %f seconds" % f.getFileDuration())
print("number of datarecords in the file: %i" % f.datarecords_in_file)
print("number of annotations in the file: %i" % f.annotations_in_file)

datarecord duration: 43199.000000 seconds
number of datarecords in the file: 43199
number of annotations in the file: 0


In [9]:
channel = 1
print("\nsignal parameters for the %d.channel:\n\n" % channel)
print("label: %s" % f.getLabel(channel))
print("samples in file: %i" % f.getNSamples()[channel])
print("samplefrequency: %f" % f.getSampleFrequency(channel))



signal parameters for the 1.channel:


label: EOG-L
samples in file: 11058944
samplefrequency: 256.000000


In [10]:
channel = 8
print("\nsignal parameters for the %d.channel:\n\n" % channel)
print("label: %s" % f.getLabel(channel))
print("samples in file: %i" % f.getNSamples()[channel])
print("samplefrequency: %f" % f.getSampleFrequency(channel))



signal parameters for the 8.channel:


label: Flow
samples in file: 1382368
samplefrequency: 32.000000


As we see Flow signal are in the 8th channel (starting with oth channel) and the number of samples is 1382368.

In [11]:
channel = 8
FlowSignal = f.readSignal(channel)
samples = 1382368
samplefreq = 32

In [12]:
datacols = ['Epoch', 'Flow']

In [13]:
# Maping Flow signals to 13 decimal place
Flow = list(map(lambda x: "%.13f" % FlowSignal[x], np.arange(samples)))

# Epochs
Epoch = list(map(lambda x: x/samplefreq, range(samples)))

In [14]:
data2_dirct = {
    'Epoch' : Epoch,
    'Flow' : Flow
}

In [15]:
data2 = pd.DataFrame(data2_dirct, columns=datacols)
print(data2)

               Epoch              Flow
0            0.00000  -0.0698558022431
1            0.03125  -0.0892652780957
2            0.06250  -0.0901808194095
3            0.09375  -0.0916456855116
4            0.12500  -0.0938429846647
5            0.15625  -0.0967727168688
6            0.18750  -0.0982375829709
7            0.21875  -0.0976882581827
8            0.25000  -0.0958571755550
9            0.28125  -0.0898146028840
10           0.31250  -0.0823071641108
11           0.34375  -0.0747997253376
12           0.37500  -0.0654612039368
13           0.40625  -0.0568551155871
14           0.43750  -0.0557564660105
15           0.46875  -0.0581368734264
16           0.50000  -0.0579537651637
17           0.53125  -0.0563057907988
18           0.56250  -0.0581368734264
19           0.59375  -0.0603341725795
20           0.62500  -0.0607003891051
21           0.65625  -0.0625314717327
22           0.68750  -0.0685740444038
23           0.71875  -0.0716868848707
24           0.75000  -0.

In [16]:
data2.to_csv('out2.csv', index=False)

## YAAAY!!!

![alt text](https://media.giphy.com/media/13SSghvwNGGW9W/giphy.gif)


## Tasks Update
    [Note to self : Complete the tasks and git commit :P]

   [x] Make a .csv file based on Hypopnea from data/apnea/data.xml (previously mesa-sleep-0001-profusion.xml)

   [x] Convert the given mesa-sleep-0001.edf file into a csv file

   [x] Make an analogy from .edf file to understand which column is flow(Flow data is sampled at 32 samples per second)

   [x] Add the first column as time or Epoch, ime should start from 0 with consecutive steps of 1/32 second (see the output file requirement) 

   [ ] Export label/EventType Stages|Stages which has a stage, startTime, and duration of a stage as csv (see the output file format.)
   
   [ ] Combine all these three files together in one file (like output/combined/mesa-sleep-0001.csv)
   
   
### Attributions 
Got a better understandind of EDF files [here](https://sleepdata.org/forum/getting-started-with-edf-files). The Documentation of [pyedflib module](https://github.com/holgern/pyedflib "Why Just Hover?? The Documentation rocks"), specially the demo on how to use pyedflib.reader :D

# Deadline Update by mBreath
![alt text](imgs/deadline.png)

![alt text](https://media.giphy.com/media/eEHgdX9bZVZHW/giphy.gif)

## Task 3 @ data processing with data/stage/mesa-sleep-0001-nsrr.xml

In [17]:
data_xml = et.parse("data/stage/mesa-sleep-0001-nsrr.xml")
data_xml

In [18]:
data3cols = ['Duration','Stages', 'Start', 'Type']
data3 = pd.DataFrame(columns=data3cols)
r = data_xml.getroot()
r.tag

'PSGAnnotation'

In [19]:
for node in r.iter("ScoredEvent"):
    duration = node.find('Duration')
    stages = node.find('EventType')
    start = node.find('Start')
    typee = node.find('EventConcept')

    if(getvalueofnode(stages) == "Stages|Stages"):
        data3 = data3.append(
        pd.Series([getvalueofnode(duration), getvalueofnode(stages), getvalueofnode(start),
                   getvalueofnode(typee)], index=data3cols),ignore_index=True)

data3.to_csv('out3.csv', index=False)

## Tasks Update
    [Note to self : Complete the tasks and git commit :P]

   [x] Make a .csv file based on Hypopnea from data/apnea/data.xml (previously mesa-sleep-0001-profusion.xml)

   [x] Convert the given mesa-sleep-0001.edf file into a csv file

   [x] Make an analogy from .edf file to understand which column is flow(Flow data is sampled at 32 samples per second)

   [x] Add the first column as time or Epoch, ime should start from 0 with consecutive steps of 1/32 second (see the output file requirement) 

   [x] Export label/EventType Stages|Stages which has a stage, startTime, and duration of a stage as csv (see the output file format.)
   
   [ ] Combine all these three files together in one file (like output/combined/mesa-sleep-0001.csv)
   
   
   